# Tarea 3 - Cifrado Moderno II
### TEL252 - Criptografía y Seguridad de la Información
#### _2do Semestre, 2023_
#### _Docente y Autor : Berioska Contreras Vargas - berioska.contreras@usm.cl_

### Nombre:
- Alejandro Cáceres
- Enrique Escalona
- Javier Martínez
- Benjamín Ramirez

* Polinomios: Pueden ser representados como una lista de coeficientes. Mediante las bibliotecas de Numpy podemos crear y modificar polinomios utilizando la clase Polynomials.  
ref: https://www.numpy.org.cn/en/reference/routines/polynomials.html

In [ ]:
import numpy as np  
import numpy.polynomial as gf  
grado = 3
gf.Polynomial.basis(grado)

* Coeficientes : Un polinomio es tratado como una serie de potencias finitas. Por lo tanto, la serie es representada en un arreglo unidimensional basado en los coeficientes del polinomio. Cada serie es ordenada ascendentemente o de menor a mayor.

In [ ]:
import numpy as np  
from numpy.polynomial import Polynomial as gf 
cf = [0,0,1]
Ax = gf.degree(cf)
mypoly = gf(cf)
print(type(mypoly))
print(mypoly.coef, mypoly.domain, mypoly.window)
print ('grado:', Ax)

z = gf.basis(deg=Ax, domain=[0,1], window=[0,1]) #

* Adición: Un polinomio permite representar un conjunto finito de enteros y por lo tanto, acepta operaciones finitas.

In [ ]:
import numpy as np  
from numpy.polynomial import Polynomial as gf 
ca = [1,1,1]
cb = [1,1,0,1]
pa = gf(ca)
pb = gf(cb)
pa + pb



* Multiplicación: Un campo $GF(x)$ es un conjunto finito de números que permite distintas operacionesa aritméticas.

In [2]:
import numpy as np  
from numpy.polynomial import Polynomial as gf 
ca = [1,1,1]
cb = np.flip([1,0,1])
pa = gf(ca)
pb = gf(cb)
pa * pb 

Polynomial([1., 1., 2., 1., 1.], domain=[-1.,  1.], window=[-1.,  1.])

* Aritmética Modular: Las operaciones suma, resta y multiplicación se realizan en conjuntos, y todas las condiciones en $GF(x)$ son satisfechas para la adición y multiplicación como expresión modular.

In [ ]:
import numpy as np
from numpy.polynomial import Polynomial as gf 
def mymod(a,b):
    return ((a + b) % 2)
ff0 = [0,0,0]
ff1 = [1,1,1]
ff2 = [1,1,1]
a, b = gf(ff0, domain=[0,1]), gf(ff1,domain=[0,1])
c = gf(ff2, domain=[0,1])
vfunc = np.vectorize(mymod)
print((a + b), '==', vfunc(ff0,ff1))
print((b + c), '<>', vfunc(ff1,ff2))

* Multiplicación en Campos Extendidos: Para cada número primo $p$ y entero positivo $n$, existe un $GF(x)$ de orden $p^n$, y su grupo multiplicativo es cíclico de orden $p^{(n-1)}$. Luego, necesitamos reducir polinomio para el orden dado.

In [36]:
import numpy as np
from numpy.polynomial import Polynomial as gf
x= gf([0,0,0], domain=[0,1], window=[0,1])
Ax= gf([1,1,1], domain=[0,1], window=[0,1])
Bx= gf([1,0,1], domain=[0,1], window=[0,1])
Cx= gf._get_coefficients(x,(Ax*Bx)) % 2 # Producto
print(gf(Cx))
Zx= gf.cutdeg(gf(Cx, domain=[0,1], window=[0,1]), deg=3) #Divisor
print(gf(Zx))
Qx= gf._get_coefficients(x,(Cx//Zx)) % 2 # Cociente
print(gf(Qx))

1.0 + 1.0 x**1 + 0.0 x**2 + 1.0 x**3 + 1.0 x**4
1.0 + 1.0 x**1 + 0.0 x**2 + 1.0 x**3
1.0 + 1.0 x**1


* Inverso Multiplicativo: Si $mcd(a,b) = 1$, los enteros $a$ y $b$ son coprimos. Este mismo principio aritmético se utiliza para los polinomios.

In [ ]:
import numpy as np
from numpy.polynomial import Polynomial as gf
x= gf([0,0,0,0,0], domain=[0,1], window=[0,1])
# x^2 
Q= gf([0,0,1,0,0], domain=[0,1], window=[0,1]) 
# x^3 + x^2 + 1
cQ= gf([1,0,1,1,0], domain=[0,1], window=[0,1]) 
# x^4+x+1
R= gf([1,1,0,0,1], domain=[0,1], window=[0,1]) 
# x + 1
cR= gf([1,1,0,0,0], domain=[0,1], window=[0,1])
iQ = gf._get_coefficients(x,(cQ*Q)) % 2 
iR = gf._get_coefficients(x,(cR*R)) % 2 
print((iQ + iR) % 2)
iRQ = gf._get_coefficients(x,(iQ+iR)) % 2 
gf(iRQ)


* Caso - Dada la interceptación de un fragmento de texto plano y flujo cifrado mediante un generador LFSR, deducir el vector de inicialización.

In [ ]:
import numpy as np
import math
def rowOperation(iv,m):
    s3 = np.flip(cts[0][0:m])
    s4, s5 = np.array([0,1,0]), np.array([1,0,1])
    ml, mr = np.matrix([s3,s4,s5]), np.matrix([[0],[1],[1]])  
    mi = ml.I # matriz inversa
    if np.matrix.sum((ml*(mi*mr)-mr)) == 0:
        return mi * ml, mr
    
pt= [1,0,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0,0,1,0,0,1,0,0,1,1,0]
pta= np.reshape(pt,(7,4)) #pta
ct= [1,0,1,1,1,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,1]
cta= np.reshape(ct,(7,4)) #cta
out= []
for i in range(0,7):
     out= np.append(out,np.bitwise_xor(pta[i],cta[i]))
cts= np.array(np.reshape(out,(4,7)), dtype=int)
#print(cts)
if math.log2(((2**3)-1)+1) == 3: m=3
echelon, coef = rowOperation(cts,m)
print(echelon)
#print(np.flip(coef.T))

**Actividad 1**: Dado el orden $GF(2^3)$ y el polinomio $x^3 + x +1$, construya de manera programática la tabla multiplicativa adjunta. Considere que, para todo valor de $i, j \in \{1, 2, 3, 4, 5, 6, 7\}$ correspondientes a los índices de la matriz $m$, se cumple, por ejemplo, la siguiente operación:  
\begin{equation*}
m[4][3] = 4 \cdot 3 = \{100\} \cdot \{011\} = x^2 \cdot (x + 1) = x^3 + x^2 = (x + 1) + x^2 = x^2 + x + 1 = \{111\} = 7
\end{equation*}

In [ ]:
import numpy as np
m = np.matrix([[1, 2, 3, 4, 5, 6, 7],
               [2, 4, 6, 3, 1, 7, 5],
               [3, 6, 5, 7, 4, 1, 2],
               [4, 3, 7, 6, 2, 5, 1],
               [5, 1, 4, 2, 7, 3, 6],
               [6, 7, 1, 5, 3, 2, 4],
               [7, 5, 2, 1, 6, 4, 3],])
m

In [8]:
import numpy as np
from numpy.polynomial import Polynomial as gf
div = gf([1,1,0,1] , domain=[0,1], window=[0,1]) # polinomio divisor
x= gf([0,0,0], domain=[0,1], window=[0,1])
matrix = np.zeros((7,7))
for i in range(1,8):
    list_i = [int(bit) for bit in bin(i)[2:]]
    list_i = list_i[::-1] # trasnformar indice i a binario
    i_pol = gf(list_i, domain=[0,1], window=[0,1])
    for j in range(1,8):
        list_j = [int(bit) for bit in bin(j)[2:]]
        list_j = list_j[::-1] # trasnformar indice j a binario
        j_pol = gf(list_j, domain=[0,1], window=[0,1])
        Cx= gf._get_coefficients(x,(j_pol*i_pol)) % 2 # Producto
        Cx_pol = gf(Cx,domain=[0,1],window=[0,1])
        coef = gf._get_coefficients(x,(Cx_pol%div)) % 2
        value = 0
        cont = 0
        for bits in coef:
            value += (2**cont)*bits
            cont += 1
        matrix[i-1][j-1]=int(value)
matrix

array([[1., 2., 3., 4., 5., 6., 7.],
       [2., 4., 6., 3., 1., 7., 5.],
       [3., 6., 5., 7., 4., 1., 2.],
       [4., 3., 7., 6., 2., 5., 1.],
       [5., 1., 4., 2., 7., 3., 6.],
       [6., 7., 1., 5., 3., 2., 4.],
       [7., 5., 2., 1., 6., 4., 3.]])

* Bits y Bytes: Desde la distribución de Python v3 es posible utilizar el tipo de dato bytes(). Este tipo de datos representa un secuencia de valores en el rango desde 0 hasta 255 (8 bits) que son almacenados de manera inalterable. 

In [ ]:
#bits
mybyte1 = int('11001000',2)
mybyte2 = int('00001000',2)
print(mybyte2 ^ mybyte1)  # a xor a
print(mybyte2 or mybyte1) # a or a
print(mybyte2 and mybyte1)  # a and a
#bytes
mybyte3 = bytes(4) #inmutable  -> help(bytes)
print(mybyte3)
myarray = [mybyte3[i] for i in mybyte3]
print (myarray)    
print (type(myarray))

* ByteArray: Complementariamente a bytes() está el tipo de datos bytearray() que sí crea un objeto alterable. La representación de los datos puede cambiar de acuerdo al sistema. Por ejemplo, el orden de los bytes cambia según el formato Endian, los valores más grandes primero (big endian), si no los pequeños primero (little endian). Otro ejemplo, sabemos que un byte no necesita ser firmado (unsigned) para representar datos en el rango positivo entre 0 y 255. Sin embargo, si fuese necesario representar valores en el rango -128 hasta +127, los bytes deben ser firmados (signed) para soportar un recorrido negativo. Ref.: https://docs.python.org/3/library/stdtypes.html

In [ ]:
#bloques
mybytestream = [255,255,255,255] 
myblock = bytearray(mybytestream)
print(myblock[:2],'y',myblock[-2:])
print(bytearray(str(myblock),'utf-8'))

#formato
k = bytes([128,192,240,255])
print(k[1:2] , '=', bytes(k[1:2]))
ki = int.from_bytes(k[1:2], byteorder='big', signed=False)
kj = int.from_bytes(k[1:2], byteorder='little', signed=True)
print(ki, '!=', kj)

* Cifrado de Bloques : Sabemos que AES opera en bytes y que utiliza múltiples tamaños de llave, así como un número variable de ciclos e incluso soporta múltiples bloques desde 128bits hasta 256bits. Sin embargo, qué sucede cuando el bloque tiende a ser pequeño? analice los eventuales riesgos del siguiente ejemplo.

In [ ]:
import numpy as np
import string
def funcionF(a,b):
    K = np.array([1,1,0,0,1,1,0,0])
    X = ((a + b) % 2) ^ K   
    return X 
def funcionR(a,b):
    A_str = ''.join(str(x) for x in a)
    B_str = ''.join(str(x) for x in b)
    return (int(A_str,2),int(B_str,2))
M = '1111000011111111'
bloques = [M[i:i+8] for i in range(0,len(M),8)]
bloqueA = np.array(list(f'{int(bloques[0],2):b}'), dtype=int)
bloqueB = np.array(list(f'{int(bloques[1],2):b}'), dtype=int)
print(bloqueA, 'y', bloqueB)
print(funcionR(bloqueA,bloqueB))
A, B = (bloqueA ^ funcionF(bloqueA,bloqueB)), (bloqueB ^ funcionF(bloqueA,bloqueB))
print(funcionR(A,B))
C1, C2 = (A ^ funcionF(A,B)), (B ^ funcionF(A,B))
print(funcionR(C1,C2))

* Doble Cifrado: Si la debilidad del algoritmo DES recae en la longitud de llave, entonces un doble cifrado qué más seguro podría ser? analice el siguiente código.

In [ ]:
def funcionW(m):
    B = [bin(M[i]) for i in range(len(M))]
    T = [B[i][2:] for i in range(len(B))]
    return T
def funcionQ(k):
    S = [bin(k[n]) for n in range(len(k))]
    V = [S[n][2:] for n in range(len(S))]
    return V
def funcionE(x,y,z):
    return ((x ^ y)^z)
K0 = bytes([254,255,254,255])  
K1 = bytes([254,255,254,255])  
M = bytes([128,192,240,255])

MX = np.zeros((4,8),dtype=int)
KX0, KX1 = np.zeros((4,8),dtype=int), np.zeros((4,8),dtype=int)
for i in range(0,4):
    for j in range (0,8):         
        t0 = funcionW(M)[i][j:j+1]
        t1, t2 = funcionQ(K0)[i][j:j+1], funcionQ(K1)[i][j:j+1]
        MX[i][j] = t0
        KX0[i][j], KX1[i][j] = t1, t2

print(funcionE(MX,KX0,KX1))

* Perfilamiento del Tiempo: Así como el efecto avalancha intenta estimar el porcentaje de difusión de los bits permutados. El perfilamiento del tiempo intenta medir la eficiencia o el tiempo que tarda la ejecución de alguna rutina, ya sea una línea o un módulo.  
Ref: http://pynash.org/2013/03/06/timing-and-profiling/

In [ ]:
import numpy as np
p = [2, 3, 7, 11, 13]
%time [x*5 for x in p] # una iteración
%timeit -n 10 [x*5 for x in p] # múltiples iteraciones

a_numpy = np.array(p)
%time [5 * a_numpy]
%timeit -n 10 [5 * a_numpy]

r = [2, 4, 6, 8, 10]
%time [sum(x) for x in zip(p,r)]

large_a, large_b = [1,2]*10**5, [2,3]*10**5
print(len(large_a),len(large_b))
%time tmp = [sum(x) for x in zip(large_a, large_b)]
print(len(tmp), tmp[:10])

large_a_np = np.array(large_a)
large_b_np = np.array(large_b)
%time large_sum = large_a_np + large_b_np
print(len(large_sum), large_sum[:10])

* Padding: En los modos de cifrado por bloques se necesita que el mensaje en texto plano tenga un valor múltiplo con respecto al tamaño del bloque, y de no ser así debemos completar el bloque para satisfacer la operación.

In [ ]:
v = np.arange(1,9,1)
mv = v.reshape((2,4))
print(mv)
np.pad(mv,(0,4),'constant', constant_values=(0))

k = bytes([254,255]) 
mk = np.array(bytearray(k),dtype=int)
print(mk)
np.pad(mk,(0,4),'constant', constant_values=(0))
#np.pad()

#### Función HASH MD5: Las funciones HASH son irreversibles y son usadas para vertificación de integridad, y para firma digital en combinación de algoritmos de llave pública. El ejemplo siguiente utiliza la función Message Digest MD5 para proteger contraseñas. MD5 genera un resumen de longitud 128 bits y fue ampliamente utilizado en protocolos de Internet, sin embargo, es vulnerable a ataques de preimagen y colisión fuerte desde 2004, y ya no es recomendable. Esta rutina se apoya en los módulos Crypto.Hash escritos en Python. El resumen obtenido se encuentra en formato hexadecimal por conveniencia. 

ref.: https://github.com/Legrandin/pycryptodome/blob/master/Doc/src/hash/hash.rst


In [ ]:
from Crypto.Hash import MD5
import pandas as pd
wl = pd.Series(['abc123', 'qwerty', 'password'])
dfw = pd.DataFrame(wl, columns=['wl'])#dfw
h = MD5.new(data=bytearray(str(dfw['wl'][0]),'utf-8')).hexdigest()
print(dfw['wl'][0]);print(h)

#### Fuerza Bruta : Es una táctica de seguridad ofensiva. Si un atacante compromete un dispositivo, entonces intentará recopilar los valores hash de las contraseñas disponibles. Sin conocer las contraseñas en texto claro, el atacante intentará sistemáticamente adivinar la contraseña utilizando mecanismos iterativos. La verificación puede ocurrir en interacción directa con el servicio, como indirectamente (offline) utilizando credenciales recopiladas, como los valores hash de las contraseñas. 
 
ref: https://attack.mitre.org/techniques/T1110/ 

In [ ]:
hl = { 'u00':   [10,'5f4dcc3b5aa765d61d8327deb882cf99','password'],
       'u01':   [20,'29edcb86ade4e0a0dea5d2c786e4ec88','v4p0r'],       
       'u02':   [30,'e99a18c428cb38d5f260853678922e03','abc123'],
       'u03':   [40,'a5f28bae5741e3a5400b62010e454cba','a1re'],
       'u04':   [50,'9ec110f58df34a0f0ade6a1fee3a2597','g4s'],
       'u05':   [60,'4f4f37f59e0502b2aa59d232f1317e59','hum0']}
dfh = pd.DataFrame(hl, index=['uid','md5','wrd']) #dfh
dfh.loc[['md5'],['u02']].to_csv('hashList.txt', header=False, index=False)
dfh.loc['wrd'].to_csv('wordList.txt', header=False, index=False)
print(open('hashList.txt','r').read())
print(open('wordList.txt','r').read())

#### Password Guessing y HashCat: Un adversario puede adivinar una credencial utilizando una lista de contraseñas comúnes y sin conocimiento previo del sistema o ambiente donde residen las contraseñas. Esta técnica es riesgosa en ataques directos ya que expone las cuentas a autenticaciones fallidas y bloqueos temporales de existir políticas de seguridad activas. La herramienta HashCat fue desarrollada por Jens Steube y Gabriele Gristina. HashCat contribuye a la auditoría de seguridad para verificar la existencia de contraseñas débiles o recuperación de contraseñas. Una ventaja de HashCat es que permite el paralelismo de procesadores GPU.
ref: https://attack.mitre.org/techniques/T1110/001/
ref: https://hashcat.net/hashcat/
ref: https://pypi.org/project/hashcat/

In [5]:
# pip install hashcat proposed by Michael Bann
#%system hashcat -V # v5.1.*
#%system hashcat -h
hArg = 'hashList.txt'; wArg = 'wordList.txt'; #cArg = open('cracked.txt','w')
!hashcat -m 0 -a 0 --potfile-disable -o cracked.txt {hArg} {wArg} 
!hashcat -m 0 -a 0 --potfile-disable -o cracked.txt 'hashList.txt' 'wordList.txt' 
print(open('cracked.txt','r').read())

hashcat (v6.2.5) starting

Started: Tue Oct 10 10:50:41 2023

                                  
Stopped: Tue Oct 10 10:50:41 2023


Already an instance C:\Users\berio\AppData\Roaming\Python\Python310\site-packages\hashcat\hashcat\hashcat.exe running on pid 5696



hashcat (v6.2.5) starting

Started: Tue Oct 10 10:50:41 2023

                                  
Stopped: Tue Oct 10 10:50:41 2023



Already an instance C:\Users\berio\AppData\Roaming\Python\Python310\site-packages\hashcat\hashcat\hashcat.exe running on pid 5696



#### Modos de Operación Directo y Combinado: La herramienta HashCat soporta al menos cinco modos de ataques; directo (0), combinado (1), fuerza bruta (3), e híbridos (6 y 7). Está disponible para Windows, Linux y Unix.  El modo directo (straight) utiliza una lista simple de palabras y comprueba cada palabra como una potencial contraseña. El modo combinado (combinator) utiliza dos listas de palabras. Cada palabra de la primera lista se adjunta a cada palabra de la segunda lista.

In [ ]:
dff = pd.DataFrame({'tf': ['password','dragon','abc','football']}, dtype=str)
dfm = pd.DataFrame({'tm': ['pa55w0rd', '1q2w3e4r','123']}, dtype=str)
dff['tf'].to_csv('words1.txt', header=False, index=False)
dfm['tm'].to_csv('words2.txt', header=False, index=False)
#fArg = 'words1.txt'; mArg = 'words2.txt'; cArg = open('cracked.txt','w')
!hashcat -m 0 -a 1 --potfile-disable -o cracked.txt {hArg} {fArg} {mArg}
print(open('cracked.txt','r').read())

#### Modos de Operación Fuerza Bruta: El modo fuerza bruta de HashCat (brute force), también referido como ataque de enmascaramiento (mask attack), es una técnica que especifíca un patrón de contraseñas, y la herramienta evalúa cada una de ellas. La siguiente es una lista de máscaras, o patrones predefinidos, y la secuencia de caracteres correspondiente:

| - | - |
|?l | abcdefghijklmnopqrstuvwxyz |
|?u | ABCDEFGHIJKLMNOPQRSTUVWXYZ |
|?d | 0123456789 |
|?h | 0123456789abcdef |
|?H | 0123456789ABCDEF |
|?s | !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ |
|?a | ?l?u?d?s |
|?b | 0x00 - 0xff

In [ ]:
# Sea una política de contraseñas: Longitud 6 caracteres, incluyendo una letra mayúscula y tres dígitos numéricos.  
dfb = pd.DataFrame({    'p':    ['Abc123'],
                        'md5':  ['61bd60c60d9fb60cc8fc7767669d40a1'] }, index=['users'])
dfb.loc[['users'],['md5']].to_csv('hashList.txt', header=False, index=False)
#hArg = 'hashList.txt'
!hashcat -m 0 -a 3 --potfile-disable -o cracked.txt {hArg} ?u?l?l?d?d?d
print(open('cracked.txt','r').read())

#### Modos de Operación Híbrido I: El modo híbrido es semejante al modo de combinación (combinator), y utiliza ya sea una lista de palabras y fuerza bruta, o viceversa.

In [ ]:
# Sea una política de contraseñas: Longitud 9 caracteres, incluyendo caractér especial y dígitos numéricos.  
dfb = pd.DataFrame({    'p':    ['abc123.7'],
                        'md5':  ['4989c47c48fa56384a3d43e42b37cd2a'] }, index=['users'])
dfb.loc[['users'],['md5']].to_csv('hashList.txt', header=False, index=False)
#hArg = 'hashList.txt'; wArg = 'wordList.txt'
#!hashcat -m 0 -a 6 --potfile-disable -o cracked.txt {hArg} {wArg} ?s?d
print(open('cracked.txt','r').read())

#### Modos de Operación Híbrido II: El modo híbrido es semejante al modo de combinación (combinator), y utiliza ya sea una lista de palabras y fuerza bruta, o viceversa.

In [ ]:
# Sea una política de contraseñas: Longitud 9 caracteres, incluyendo un caractér especial y dígitos numéricos.  
dfb = pd.DataFrame({    'p':    ['7.abc123'],
                        'md5':  ['3bf76bc988b14e0b80413fae13fd2aae'] }, index=['users'])
dfb.loc[['users'],['md5']].to_csv('hashList.txt', header=False, index=False)
#hArg = 'hashList.txt'; wArg = 'wordList.txt'
#!hashcat -m 0 -a 7 --potfile-disable -o cracked.txt {hArg} ?d?s {wArg}
print(open('cracked.txt','r').read())

#### Reglas de HashCat: Complementariamente a los modos de operación, HashCat también soporta reglas de permutación de palabras. Las reglas mutan un listado de palabras de acuerdo a las convenciones designadas. Ejemplos: reemplazar caracteres alfabéticos por dígitos numéricos, añadir un número o caractér especial, revertir el orden de una palabra, forzar a mayúscula el primer dígito alfabético, entre otros. 

ref.: https://github.com/hashcat/hashcat/blob/master/rules/best64.rule

In [ ]:
dfrule = pd.DataFrame({'leet': ['## leetify','so0','si1','se3','sa4']}, dtype=str)
dfrule['leet'].to_csv('base64.rule', header=False, index=False) 
dfb = pd.DataFrame({    'p':    ['P4ssw0rd'],
                        'md5':  ['d41e98d1eafa6d6011d3a70f1a5b92f0'] }, index=['users'])
dfb.loc[['users'],['md5']].to_csv('hashList.txt', header=False, index=False)
hArg = 'hashList.txt'; wArg = 'wordList.txt'; rArg ='base64.rule' 
!hashcat -m 0 -a 0 --potfile-disable {hArg} {wArg} -r {rArg}

### Actividad 2: Ha ocurrido un incidente de ciberseguridad en UTFSM!. De acuerdo a las indagaciones un desconocido publicó información en el sitio 'pastebin' accesible para todo Internet. La publicación parece ser un volcado de valores hash de diversa longitud y se desconoce el sistema o servicio comprometido. Indague las funciones hash implicadas en las evidencias siguientes, y demuestre si es posible descartar el uso de contraseñas débiles.
ref.: https://www.dailymail.co.uk/sciencetech/article-2331984/Think-strong-password-Hackers-crack-16-character-passwords-hour.html

In [ ]:
# Data Breach at UTFSM by Eve.
#cf24437db56726aa69872ea420a17063
#d45a6b1e2c56bc6a30aa3ec2e89aa7dbfcb806d5
#6cd989f3330f20cf9ccecc99f32830734d0f981135281e4b728febae767928b1
#fd112fedf8ede379b9bc268a89cb686020ec874df073314f7f7cb8977da0c492de66115792b2f2c6bba48d00b531776de3804207e828affb945739bd13ec6b2c

Lo publicado consiste en un listado de contraseñas hash, con una longitud de entre 32 y 64 caracteres. Analizando, se puede determinar que se usaron las siguientes funciones hash:

* MD5: longitud de 32 caracteres.
* SHA-1: longitud de 40 caracteres.
* SHA-256: longitud de 64 caracteres.

Para saber si se usaron contraseñas débiles, debemos analizar las funciones hash utilizadas:

* MD5: función hash obsoleta que ya no se considera segura. Un ataque de fuerza bruta contra MD5 puede descifrar una contraseña de 8 caracteres en unos segundos. Dado que la entrada siempre produce la misma salida, puede ocurrir que dos archivos distintos compartan el mismo hash (colisión MD5).

* SHA-1: Produce un hash de 160 bits, y al ser similar a MD5, cuando este fue vulnerado, era cuestión de tiempo para que le pasara lo mismo. De hecho, en el año 2017, se descubrió un ataque de colisión contra este algoritmo. Este ataque se puede llevar a cabo con entradas personalizadas, y no solo colisiones accidentales, lo que permite a los atacantes apuntar a ciertos archivos para duplicarlos. El algoritmo utilizado actualmente para lograr esto tiene una complejidad de 2^69. Esto hoy requiere mucho poder computacional, pero... ¿Y mañana?

Los dos algoritmos anteriores ya fueron vulnerados y no es recomendado su uso.

* SHA-256: ¿Es posible vulnerar SHA-256? Lograr esto, por definición, es encontrar dos mensajes m1 y m2, diferentes de tal manera que sus funciones hash sean idénticas, de esta manera: sha-256(m1) = sha-256(m2). En teoría es posible, asumiendo suficiente tiempo, dinero y, por ende, recursos de cómputo.

Si consideramos que los hashes publicados son contraseñas, para los primeros dos cifrados es posible que sean contraseñas débiles, pero como estos algoritmos de encriptación ya fueron vulnerados, no tenemos la certeza, pero sabemos que ya no son seguros para proteger claves. Por otro lado, tenemos SHA-256; aunque este algoritmo no fue creado para proteger claves (rara vez tenemos una clave de más de 12 caracteres), almacenar 32 bytes como resumen de mi clave de Instagram es un desperdicio de espacio, es el equivalente a matar una mosca con un cañón. Este algoritmo fue diseñado para crear huellas resumen de ficheros muy grandes, donde el secreto no es necesario y asegurar que no se haya cambiado ni siquiera un bit.

Por otro lado, SHA-256 es demasiado eficiente; una buena función hash para contraseñas debe ser "lenta" por diseño, lo más lenta posible mientras sea utilizable. De esa manera se asegura que no se pueda vulnerar mediante una búsqueda en diccionarios o por fuerza bruta.

Es por estas razes que ni MD5, SHA-1 y SHA-256 son buenos algoritmos para guardar contraseñas de forma segura, ya que no fueron diseñados para eso. Eso implica, además, que no necesariamente las contraseñas usadas eran débiles. Podría vulnerar los dos primeros mediante fuerza bruta con mi PC (probablemente), pero para el último necesito mucho poder de cómputo, pero no es imposible.

### Fin de la Tarea!!!